In [ ]:
import pandas as pd
import numpy as np
import os.path
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from PIL import Image

%matplotlib inline
from keras.applications import ResNet50
from keras.optimizers import Adam
import cv2
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

In [ ]:
train = pd.read_csv('/kaggle/input/hackerearth-deep-learning-challenge-holidayseason/dataset/train.csv')
train.head()

In [ ]:
pd.value_counts(train['Class']).plot.bar()
plt.xlabel('Class')
plt.ylabel('Frequency')
train['Class'].value_counts()

Class_map={'Miscellaneous':0,'Christmas_Tree':1,'Jacket':2,'Candle':3,'Airplane':4,'Snowman':5}
inverse_map={0:'Miscellaneous',1:'Christmas_Tree',2:'Jacket',3:'Candle',4:'Airplane',5:'Snowman'}
train['Class']=train['Class'].map(Class_map)

In [ ]:
TRAIN_PATH = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train/'
TEST_PATH = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/'

In [ ]:
IMG_SIZE = 64

def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    return img

In [ ]:
train_img = []
for img_name in tqdm(train['Image'].values):
    train_img.append(read_img(TRAIN_PATH + img_name ))
    
x_train = np.array(train_img, np.float32) / 255
print(x_train.shape)

In [ ]:
label_list = train['Class'].tolist()
label_numeric = {k: v+1 for v, k in enumerate(set(label_list))}
y_train = [label_numeric[k] for k in label_list]
y_train = np.array(y_train)

In [ ]:
y_train = to_categorical(y_train)
print(y_train.shape)

In [ ]:
base_model =   applications.ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [ ]:
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dropout(0.4))
add_model.add(Dense(128, activation='relu'))
add_model.add(Dropout(0.3))
add_model.add(Dense(y_train.shape[1], activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

In [ ]:
batch_size = 64      
epochs = 20                

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1, 
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_datagen.fit(x_train)

In [ ]:
history = model.fit_generator( train_datagen.flow(x_train, y_train, batch_size=batch_size),
                               steps_per_epoch=x_train.shape[0] // batch_size,
                               epochs=epochs)

In [ ]:
import gc
gc.collect()

In [ ]:
test_images = os.listdir(TEST_PATH)
test_img = []
for image in test_images:
    test_img.append(read_img(TEST_PATH + image))

In [ ]:
x_test = np.array(test_img, np.float32) / 255
predictions = model.predict(x_test)

In [ ]:
predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in label_numeric.items()}
pred_labels = [rev_y[k] for k in predictions]

In [ ]:
sub = pd.DataFrame({'Image': test_images, 'Class': pred_labels})
sub.to_csv('result.csv', index = False)
sub.head()